In [2]:
import pandas as pd
import numpy as np

import findspark
findspark.init()
from pyspark import SparkContext

sc = SparkContext("local[*]", "Name of the Program")



25/02/18 19:53:46 WARN Utils: Your hostname, jalvarez resolves to a loopback address: 127.0.1.1; using 10.166.129.52 instead (on interface wlo1)
25/02/18 19:53:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/18 19:53:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
df = pd.read_csv('data_ok.csv',sep=' ',header=None)
df

,0,1,2,3,4,5,6,7,8,9,...,201,202,203,204,205,206,207,208,209,210
0,18.0,1526.0,12.0,8377.0,1.0,7.0,1460.0,240.0,38.0,46336.0,...,298.469632,639.500000,1271.0,0.0,475.735834,310.000000,703.0,101.0,202.278686,0
1,21.0,960.0,28.0,29632.0,1.0,7.0,1460.0,407.0,38.0,92928.0,...,13.323166,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0
2,45.0,960.0,81.0,107392.0,1.0,7.0,1460.0,407.0,38.0,223232.0,...,8.639085,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0
3,66.0,960.0,137.0,188032.0,1.0,7.0,1460.0,407.0,38.0,300800.0,...,6.822344,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0
4,92.0,960.0,188.0,261472.0,1.0,7.0,1460.0,407.0,38.0,451328.0,...,5.860381,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,1042.0,11341.0,2842.0,3928201.0,1.0,7.0,1460.0,1230.0,93.0,1327104.0,...,245.429254,10610.333333,22434.0,186.0,6479.234797,2633.444444,10168.0,111.0,3399.864119,0
4996,1065.0,11341.0,2896.0,4002453.0,1.0,7.0,1460.0,1230.0,93.0,1327104.0,...,243.177400,10610.333333,22434.0,186.0,6479.234797,2633.444444,10168.0,111.0,3399.864119,0
4997,1086.0,11341.0,2952.0,4080893.0,1.0,7.0,1460.0,1230.0,93.0,1327104.0,...,240.823419,10610.333333,22434.0,186.0,6479.234797,2633.444444,10168.0,111.0,3399.864119,0
4998,1108.0,11341.0,3007.0,4157673.0,1.0,7.0,1460.0,1230.0,93.0,1327104.0,...,238.576047,10610.333333,22434.0,186.0,6479.234797,2633.444444,10168.0,111.0,3399.864119,0


In [3]:
df=np.array(df)
df

array([[1.80000000e+01, 1.52600000e+03, 1.20000000e+01, ...,
        1.01000000e+02, 2.02278686e+02, 0.00000000e+00],
       [2.10000000e+01, 9.60000000e+02, 2.80000000e+01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.50000000e+01, 9.60000000e+02, 8.10000000e+01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [1.08600000e+03, 1.13410000e+04, 2.95200000e+03, ...,
        1.11000000e+02, 3.39986412e+03, 0.00000000e+00],
       [1.10800000e+03, 1.13410000e+04, 3.00700000e+03, ...,
        1.11000000e+02, 3.39986412e+03, 0.00000000e+00],
       [1.12600000e+03, 1.24620000e+04, 3.06600000e+03, ...,
        1.11000000e+02, 4.76878318e+03, 0.00000000e+00]],
      shape=(5000, 211))

## Load data

## Ej1

In [4]:
array=df[:,38]
rdd = sc.parallelize(array)
suma = rdd.reduce(lambda x,y: x+y)
media = suma / rdd.count()

rdd_2 = rdd.map(lambda x: (x, media))

restas = rdd_2.map(lambda x: (x[0]-x[1])**2)
sumas = restas.reduce(lambda x,y: x+y)
varianza = sumas / rdd.count()

print(f'Medias: {media}. Varianzas: {varianza}')
print(f'Real. Media: {rdd.mean()}. Varianza: {rdd.stdev()**2}')

Medias: 1058.840969225582. Varianzas: 3760978.8000936448
Real. Media: 1058.8409692255825. Varianza: 3760978.800093644


## Ej 2

In [5]:
rdd = sc.parallelize(df)
suma = rdd.reduce(lambda x,y: x+y)
media = suma / rdd.count()

rdd_2 = rdd.map(lambda x: (x, media))

restas = rdd_2.map(lambda x: (x[0]-x[1])**2)
sumas = restas.reduce(lambda x,y: x+y)
varianza = sumas / rdd.count()


#Se imprimen los 3 primeros para comprobar
print(f'Medias: {media[:3]}. Varianzas: {varianza[:3]}')
print(f'Real. Media: {rdd.mean()[:3]}. Varianza: {(rdd.stdev()**2)[:3]}')


Medias: [ 427.8384 8544.5638 1006.7712]. Varianzas: [4.01818166e+05 3.16837334e+09 2.95960564e+06]
Real. Media: [ 427.8384 8544.5638 1006.7712]. Varianza: [4.01818166e+05 3.16837334e+09 2.95960564e+06]


## Ej 3

In [39]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ReduceByKeyExample").getOrCreate()

# Ruta del archivo CSV
archivo_csv = "data_ok.csv"

# Leer el archivo como un RDD
rdd = spark.sparkContext.textFile(archivo_csv)
total_filas = float(rdd.count())

# Transformar cada fila en una lista de tuplas (columna, valor)
rdd_tuplas = rdd.flatMap(lambda x: enumerate(x.split()))

# Convertir en un RDD de pares (columna, valor)
rdd_pares = rdd_tuplas.map(lambda x: (x[0], float(x[1])))  # 🔥 Corregido

# Aplicar reduceByKey para sumar los valores por columna
rdd_reducido = rdd_pares.reduceByKey(lambda x, y: x + y)
 

rdd_media = rdd_reducido.map(lambda x: (x[0], (x[1]/total_filas)))

# Mostrar solo las primeras 5 filas en lugar de collect()
media = rdd_media.collect()
media=sorted(media,key=lambda x: x[0])

rdd_potencia = rdd_pares.map(lambda x: (x[0], ((x[1]-media[x[0]][1])**2)))
varianza = rdd_potencia.reduceByKey(lambda x, y: x + y)
std=varianza.map(lambda x: (x[0], (x[1]/total_filas)))
std=std.collect()
std=sorted(std,key=lambda x: x[0])
[(i, media[i][1], std[i][1] ) for i in range(len(media))]

[(0, 427.8384, 401818.16628544027),
 (1, 8544.5638, 3168373338.3683057),
 (2, 1006.7712, 2959605.6432505627),
 (3, 1409608.9152, 5883755024032.205),
 (4, 1.0, 0.0),
 (5, 6.6542, 2.3010223599999864),
 (6, 1460.0, 0.0),
 (7, 762.3126, 313332.59728123934),
 (8, 140.2394, 19828.36648764016),
 (9, 506626.9196, 345513253204.2377),
 (10, 28973.7058, 6514263.902446111),
 (11, 0.0108, 0.03228335999999504),
 (12, 0.9462, 0.05090556000000112),
 (13, 7.5006, 3.9563996399999914),
 (14, 1411.0728, 14631.804300159674),
 (15, 3307.1342, 19686496.994590227),
 (16, 198.7724, 103958.43979824062),
 (17, 835825.356, 12358118332394.264),
 (18, 43811.7826, 435068936.0129385),
 (19, 0.0, 0.0),
 (20, 238.93478482702486, 114088.27606048658),
 (21, 224802.51876693306, 151781717098.3398),
 (22, 16.71908086065675, 3110.345729247053),
 (23, 2519.3622, 19567004.17901126),
 (24, 0.0, 0.0),
 (25, 114.84695369961145, 105249.50629016865),
 (26, 36.99778951411964, 11237.074611699474),
 (27, 2531.919, 19550727.745639034),

## Ej 4

In [ ]:
rdd = sc.parallelize(df)
suma = rdd.reduce(lambda x,y: x+y)
media = suma / rdd.count()

rdd_2 = rdd.map(lambda x: (x, media))

restas = rdd_2.map(lambda x: (x[0]-x[1])**2)
sumas = restas.reduce(lambda x,y: x+y)
varianza = np.sqrt(sumas / rdd.count())



In [10]:
rdd_media_final.collect()

[(0, 427.8384),
 (2, 1006.7712),
 (4, 1.0),
 (6, 1460.0),
 (8, 140.2394),
 (10, 28973.7058),
 (12, 0.9462),
 (14, 1411.0728),
 (16, 198.7724),
 (18, 43811.7826),
 (20, 238.93478482702486),
 (22, 16.71908086065675),
 (24, 0.0),
 (26, 36.99778951411964),
 (28, 0.0),
 (30, 31.5279869144895),
 (32, 2.0964),
 (34, 4113.869304378967),
 (36, 182.2178),
 (38, 1058.8409692255857),
 (40, 402.8786),
 (42, 428.1766),
 (44, 1007.4334),
 (46, 1.0),
 (48, 1460.0),
 (50, 141.7162),
 (52, 28973.7058),
 (54, 0.9526),
 (56, 1420.7528),
 (58, 199.1446),
 (60, 44052.8954),
 (62, 239.52861091135205),
 (64, 16.791656401665527),
 (66, 0.0),
 (68, 37.2071994155398),
 (70, 0.0),
 (72, 31.728554784238863),
 (74, 2.223),
 (76, 4118.800382986885),
 (78, 182.7526),
 (80, 1060.040436616305),
 (82, 403.1364),
 (84, 428.5226),
 (86, 1008.0882),
 (88, 1.0),
 (90, 1460.0),
 (92, 143.0178),
 (94, 28973.7058),
 (96, 0.9526),
 (98, 1420.7528),
 (100, 202.0514),
 (102, 44044.4058),
 (104, 239.91163465734795),
 (106, 16.9868